# Análisis Predictivo: Fuga de Clientes en Telecomunicaciones

Este cuaderno tiene como objetivo explorar un caso real de predicción de fuga de clientes usando un árbol de decisión.

## Objetivo
Reflexionar sobre el uso de modelos predictivos y prescriptivos para tomar decisiones empresariales.

📌 **Nota:** No es necesario programar. Simplemente ejecuta las celdas y reflexiona sobre las preguntas propuestas.

In [ ]:
# Cargar librerías
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

# Cargar datos desde GitHub
url = "https://raw.githubusercontent.com/taruntiwarihp/dataSets/master/telecom_churn.csv"
df = pd.read_csv(url)
df.head()

,State,Account length,Area code,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls,Churn
0,KS,128,415,No,Yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,No,Yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,No,No,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,408,Yes,No,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,415,Yes,No,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False


## ¿Qué representan estos datos?
- Cada fila representa un cliente.
- Las columnas muestran información sobre su uso de servicios, gastos y si se dio de baja (`Churn`).

### 💬 Reflexión:
¿Qué variables crees que podrían influir más en que un cliente se dé de baja?

- Customer Service Calls:

Un alto número de llamadas al servicio al cliente puede reflejar insatisfacción.

- Account length:

Los clientes con menor antigüedad suelen tener mayor probabilidad de darse de baja en comparación con los clientes de larga duración.

- Voice Mail Plan y Number vmail messages:**

Contar con un plan de buzón de voz y recibir pocos mensajes podría indicar una baja interacción del cliente con la empresa.



In [ ]:
# Preprocesamiento mínimo (automático)
df.drop(["State", "Area code", "Phone"], axis=1, inplace=True, errors='ignore')
df["Churn"] = df["Churn"].astype(int)
df = pd.get_dummies(df, drop_first=True)

X = df.drop("Churn", axis=1)
y = df["Churn"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Entrenamiento y predicción con árbol de decisión
model = DecisionTreeClassifier(max_depth=5, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.98      0.96       566
           1       0.88      0.67      0.76       101

    accuracy                           0.94       667
   macro avg       0.91      0.83      0.86       667
weighted avg       0.93      0.94      0.93       667



### 💬 Reflexión:
**- ¿El modelo predice bien los clientes que se dan de baja?**

- Recall (Sensibilidad o Exhaustividad) para la clase "1" es de 0.67: Este valor indica que el modelo identifica correctamente al 67% del total de clientes que realmente se dieron de baja. Por consiguiente, un 33% de los clientes que causaron baja no fueron detectados por el modelo.
- Precision para la clase "1" es de 0.88: Esta métrica significa que, cuando el modelo predice que un cliente se dará de baja, dicha predicción es correcta en el 88% de los casos. El 12% restante de las predicciones de baja son falsas alarmas.
- F1-score para la clase "1" es de 0.76: Siendo la media armónica de la precisión y el recall, proporciona una medida equilibrada del rendimiento para esta clase.

**- ¿Qué errores comete?**

- Falsos Negativos (FN): Corresponden a clientes que efectivamente se dieron de baja, pero que el modelo clasificó erróneamente como no propensos a la baja. Dado un recall de 0.67, la tasa de falsos negativos es del 33% (1 - 0.67). Si el conjunto de soporte para la clase "1" es de 101 clientes, esto significa que aproximadamente 33 clientes que causaron baja no fueron identificados por el modelo (101 * 0.33 ≈ 33).
- Falsos Positivos (FP): Se refieren a clientes que el modelo predijo que se darían de baja, pero que en realidad permanecieron activos. Con una precisión de 0.88, el 12% de las predicciones de baja son falsos positivos (1 - 0.88). Para ilustrar, si el modelo realizó X predicciones de baja, y 0.88 * X fueron verdaderos positivos (67 en este caso, ya que 67 / 0.88 ≈ 76 predicciones de baja), entonces aproximadamente 9 clientes (76 - 67) fueron incorrectamente marcados como bajas potenciales.

**- ¿Qué consecuencias tendría para la empresa?**

Consecuencias de los Falsos Negativos:  
- Pérdida de clientes e ingresos.  
- Oportunidades de retención desaprovechadas.

Consecuencias de los Falsos Positivos:  
- Costes innecesarios en retención.  
- Posible molestia a clientes que no pensaban irse.

**- ¿Preferirías minimizar los falsos positivos o los falsos negativos?**

- Creo que es preferible minimizar los falsos negativos, ya que resulta es más costoso perder un cliente por no haberlo detectado a tiempo.

## De la predicción a la acción: Análisis prescriptivo

Supón que detectas que clientes con muchas llamadas al servicio de atención tienden a darse de baja.

### 💬 Reflexión:

**- ¿Qué acción podrías recomendar a la empresa?**

- Implementaría un sistema de Alerta de Churn por Fricción en soporte

1. Análisis de Causa Raíz Automatizado: Utilizar NLP (Procesamiento de Lenguaje Natural) sobre las transcripciones/resúmenes de las llamadas para identificar automáticamente los motivos recurrentes y el sentimiento de los clientes que llaman mucho.
2. Segmentación de Riesgo Dinámica: Combinar la frecuencia de llamadas, los motivos identificados por NLP y el sentimiento con el valor del cliente (CLTV) para priorizar a los clientes de alto riesgo y alto valor.
3. Acción Proactiva Personalizada y Desencadenada por Datos:
    - Para problemas técnicos recurrentes: Desencadenar una oferta de revisión técnica proactiva o un upgrade temporal a un servicio más estable.
    - Para frustración por procesos/facturación: Escalar automáticamente a un equipo especializado de retención con la autoridad para ofrecer soluciones rápidas (ej. créditos, descuentos).

**- ¿Cómo podrías personalizar la oferta o el servicio?**

- Utilizar NLP (Procesamiento de Lenguaje Natural) en las transcripciones de las llamadas para diagnosticar automáticamente el problema raíz específico de cada cliente. Luego, ofrecer soluciones altamente personalizadas y proactivas:

1. Si el problema es técnico recurrente: Ofrezca una revisión técnica proactiva, un upgrade a un equipo/servicio más estable, o acceso directo a soporte técnico especializado.
2. Si es dificultad de uso o comprensión: Provea guías interactivas personalizadas, tutoriales específicos para las funcionalidades problemáticas, o una sesión de ayuda individualizada.
3. Si es insatisfacción con el plan/facturación: Proponga un cambio a un plan más adecuado a su consumo real (inferido de sus datos) o envíe una explicación simplificada y personalizada de su factura.